In [1]:
import numpy as np
import pandas as pd
from scipy import stats

## Latitude & Longitude

In [110]:
ldf = pd.read_excel("B:\GroundwaterRech\GithubRepo\Restart\Forecast_new\LatLong_mapping.xlsx")
ldf.drop(['Unnamed: 0'], inplace=True, axis=1)

In [111]:
ldf

,New_Longitude,New_Latitude,lon,lat
0,74.75,31.25,74.80,31.30
1,75.00,31.25,75.05,31.30
2,74.75,31.50,74.80,31.55
3,75.00,31.50,75.05,31.55
4,75.25,31.50,75.30,31.55
...,...,...,...,...
60,75.75,30.25,75.80,30.30
61,76.00,30.25,76.05,30.30
62,75.50,30.50,75.55,30.55
63,75.75,30.50,75.80,30.55


In [112]:
LATITUDES = np.array(ldf['lon'].unique())
LONGITUDES = np.array(ldf['lat'].unique())

# Bias Correction

In [60]:
def mean_bias_correction(observed_data, modeled_data):
    # Calculate the mean bias
    mean_observed = np.mean(observed_data)
    mean_modeled = np.mean(modeled_data)
    bias = mean_observed - mean_modeled

    # Apply the bias adjustment
    corrected_data = modeled_data + bias
    print(bias)
    return corrected_data

In [61]:
def quantile_mapping(observed_data, modeled_data):
    # Calculate CDFs of observed and modeled data
    observed_cdf = np.cumsum(observed_data) / np.sum(observed_data)
    modeled_cdf = np.cumsum(modeled_data) / np.sum(modeled_data)

    # Perform quantile mapping
    mapped_data = np.interp(modeled_cdf, observed_cdf, observed_data)
    return mapped_data

In [62]:
# non parametric robust quantile mapping
def non_param_quantile_mapping(observed_data, modeled_data):
    reference_median = np.median(observed_data)
    reference_iqr = stats.iqr(observed_data)
    mapped_median = np.median(modeled_data)
    mapped_iqr = stats.iqr(modeled_data)

    reference_quantiles = stats.norm.ppf(np.linspace(0, 1, 101), loc=reference_median, scale=reference_iqr / 1.349)
    mapped_quantiles = stats.norm.ppf(np.linspace(0, 1, 101), loc=mapped_median, scale=mapped_iqr / 1.349)

    corrected_mapped_data = np.interp(modeled_data, mapped_quantiles, reference_quantiles)
    return corrected_mapped_data

In [63]:
def npq_mapping(observed_data, modeled_data):
    reference_quantiles = np.percentile(observed_data, np.linspace(0, 100, 101))
    mapped_quantiles = np.percentile(modeled_data, np.linspace(0, 100, 101))
    corrected_mapped_data = np.interp(modeled_data, mapped_quantiles, reference_quantiles)
    return corrected_mapped_data

In [64]:
def distribution_mapping(observed_data, modeled_data):
    reference_quantiles = np.percentile(observed_data, np.linspace(0, 100, 101))
    mapped_quantiles = np.percentile(modeled_data, np.linspace(0, 100, 101))
    corrected_mapped_data = np.interp(modeled_data, mapped_quantiles, reference_quantiles)
    return corrected_mapped_data

In [65]:
def local_intensity_scaling(observed_data, modeled_data):
    scaling_factor = observed_data / modeled_data
    bias_corrected_precipitation = modeled_data * scaling_factor
    print(scaling_factor, np.mean(scaling_factor))
    return bias_corrected_precipitation


In [66]:
def calculate_nse(observed, predicted):
    observed = np.array(observed)
    predicted = np.array(predicted)
    mean_observed = np.mean(observed)
    sse = np.sum((observed - predicted) ** 2)
    sv = np.sum((observed - mean_observed) ** 2)
    nse = 1 - (sse / sv)
    print(f'NSE: {nse}')

## Precipitation
##### Historical

In [147]:
def conv_precip(df, start_year, end_year):
    df['time'] = pd.to_datetime(df['time'])
    df['year'] = df['time'].dt.year
    df = df[df['time'].dt.month.between(6, 10)]
    df = df[df['time'].dt.year.between(start_year, end_year)]
    result = df.groupby(['lat', 'lon', 'year'])['pr'].sum().reset_index()
    result['pr'] = (86400 * result['pr']) / 5
    # df['month'] = df['time'].dt.month
    # avg_monthly_temp = df.groupby(['lat', 'lon', 'year'])['pr'].mean().reset_index()
    merged_df = ldf.merge(result, on=['lon', 'lat'], how='left')
    pivot_df = merged_df.pivot(index=['New_Latitude', 'New_Longitude'], columns='year', values='pr').reset_index()
    return pivot_df

In [149]:
df = pd.read_csv("B:\GroundwaterRech\GithubRepo\Restart\Dataset\historical\historical\pr.csv")
df.drop(['Unnamed: 0'], inplace=True, axis=1)
conv_df = conv_precip(df, 1986, 2015)
conv_df

year,New_Latitude,New_Longitude,1986,1987,1988,1989,1990,1991,1992,1993,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,29.75,75.25,55.305527,44.197033,41.181970,44.574903,36.993112,43.369195,50.767606,65.745030,...,43.220520,53.354416,63.909463,65.035362,21.743422,61.500116,38.119362,53.153770,52.600637,67.856884
1,29.75,76.00,69.139285,55.781457,49.902087,49.923606,58.058920,64.614318,65.785716,83.791145,...,50.974307,69.032763,79.742823,82.207219,26.470188,86.473225,53.407016,62.853917,65.645310,89.659905
2,30.00,74.75,47.457414,37.082299,34.359797,39.745148,26.845057,32.008821,41.272354,57.040431,...,40.278921,43.465178,56.145299,57.672667,18.401412,44.616281,28.528678,47.656226,45.724909,56.431510
3,30.00,75.00,48.997724,38.250173,35.072353,40.473921,28.782369,34.167767,43.334149,58.751364,...,41.987177,45.507090,57.533638,58.991175,18.865833,47.743339,30.404040,49.202131,46.757545,58.152887
4,30.00,75.25,54.616990,43.378701,39.129151,43.330003,37.125974,42.845308,49.860564,66.754153,...,45.451580,52.749246,64.323512,66.565109,21.143854,59.071580,37.272781,53.615025,52.742813,68.125707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,32.00,75.00,53.844115,49.845330,41.161282,37.787706,48.298174,52.628881,46.552825,91.926430,...,56.330707,43.290823,81.172200,89.593534,16.613050,42.781700,53.931237,59.108375,77.976837,97.232573
61,32.00,75.25,55.253140,51.052919,42.575997,43.545687,48.260862,59.906883,49.050468,91.131603,...,58.649986,45.843549,80.150102,87.335237,18.826974,52.224097,54.122034,58.515388,78.180403,93.844463
62,32.00,75.50,56.662165,52.260508,43.990712,49.303667,48.223550,67.184884,51.548112,90.336776,...,60.969265,48.396275,79.128003,85.076941,21.040898,61.666493,54.312830,57.922401,78.383969,90.456352
63,32.00,75.75,58.071191,53.468097,45.405427,55.061648,48.186237,74.462886,54.045755,89.541949,...,63.288545,50.949001,78.105905,82.818644,23.254822,71.108889,54.503627,57.329413,78.587535,87.068241


##### Forcasted PR

In [151]:
df2 = pd.read_csv(
    "B:\GroundwaterRech\GithubRepo\Restart\Dataset\Final forecasted data 2020-2040\Final forecasted data 2020-2040\SSP126\pr.csv")
df2.drop(['Unnamed: 0'], inplace=True, axis=1)
conv_df = conv_precip(df2, 2020, 2035)
conv_df

year,New_Latitude,New_Longitude,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
0,29.75,75.25,34.128848,57.390215,45.999076,57.278152,61.529762,34.517298,49.260005,56.382877,54.464653,69.381700,49.610066,58.560142,52.897555,56.530459,69.566694,60.509610
1,29.75,76.00,45.305026,69.708367,56.665623,73.792224,73.844339,36.878286,63.278110,76.876443,66.315481,79.168259,59.703003,73.638028,59.702633,75.848994,92.467179,79.088390
2,30.00,74.75,26.945124,50.502757,38.228411,48.838723,54.071771,31.428321,41.992901,45.138257,48.724162,64.911190,42.022072,52.439404,50.068805,44.901912,58.592399,50.537179
3,30.00,75.00,28.073660,52.077518,40.318053,50.636100,55.687502,31.738523,43.923193,47.504836,50.219763,68.607793,43.589903,54.799492,52.313836,47.084491,61.484115,53.389154
4,30.00,75.25,33.034581,58.042674,45.412354,58.047304,61.113456,33.044409,50.416799,56.626991,55.821922,73.582706,48.386407,61.458305,55.544467,55.340740,72.164651,61.887421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,32.00,75.00,32.930563,79.939319,52.448432,81.797791,93.282329,29.533070,72.044680,78.634200,58.342655,55.511599,66.578201,79.311571,50.761262,56.861717,97.953719,66.533026
61,32.00,75.25,36.292687,82.644371,53.710920,80.760833,95.475858,31.680330,69.598578,80.587898,60.637995,57.944059,66.865035,77.842493,52.083485,56.945882,94.741718,67.617312
62,32.00,75.50,39.654811,85.349423,54.973408,79.723874,97.669386,33.827590,67.152476,82.541597,62.933335,60.376520,67.151869,76.373415,53.405707,57.030046,91.529717,68.701598
63,32.00,75.75,43.016935,88.054474,56.235896,78.686916,99.862915,35.974849,64.706375,84.495295,65.228675,62.808980,67.438703,74.904338,54.727930,57.114211,88.317716,69.785884


In [152]:
conv_df.to_csv('B:\GroundwaterRech\GithubRepo\Restart\Forecast_new\SSP126\pet_SSP126.csv\pr_SSP126.csv', index=False)

# Temperature

### Min

In [137]:
# def conv_temp(df, start_year, end_year,var):
#     df['tasmin'] = df['tasmin'] - 273.15
#     df['time'] = pd.to_datetime(df['time'])
#     df['year'] = df['time'].dt.year
#     df = df[df['time'].dt.month.between(6, 10)]
#     df = df[df['time'].dt.year.between(start_year, end_year)]
#     df['month'] = df['time'].dt.month
#     avg_monthly_temp = df.groupby(['lat', 'lon', 'year'])['tasmin'].mean().reset_index()
#     merged_df = ldf.merge(avg_monthly_temp, on=['lon', 'lat'], how='left')
#     pivot_df = merged_df.pivot(index=['New_Latitude', 'New_Longitude'], columns='year', values='tasmin').reset_index()
#     return pivot_df

In [138]:
def conv_temp(df, start_year, end_year, var):
    df[var] = df[var] - 273.15
    df['time'] = pd.to_datetime(df['time'])
    df['year'] = df['time'].dt.year
    df = df[df['time'].dt.month.between(6, 10)]
    df = df[df['time'].dt.year.between(start_year, end_year)]
    df['month'] = df['time'].dt.month
    avg_monthly_temp = df.groupby(['lat', 'lon', 'year'])[var].mean().reset_index()
    merged_df = ldf.merge(avg_monthly_temp, on=['lon', 'lat'], how='left')
    pivot_df = merged_df.pivot(index=['New_Latitude', 'New_Longitude'], columns='year', values=var).reset_index()
    return pivot_df

In [131]:
df = pd.read_csv("B:\GroundwaterRech\GithubRepo\Restart\Dataset\historical\historical\\tmin.csv")
df.drop(['Unnamed: 0'], inplace=True, axis=1)
df.drop(['height'], inplace=True, axis=1)

In [132]:
conv_df = conv_temp(df, 1986, 2019)
conv_df

year,New_Latitude,New_Longitude,1986,1987,1988,1989,1990,1991,1992,1993,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,29.75,75.25,22.439344,24.513336,24.307088,23.850769,23.990606,24.857390,23.531693,23.679141,...,24.731911,24.992275,24.594872,23.328115,25.335285,24.709703,24.875445,24.320486,24.376478,24.051829
1,29.75,76.00,21.443014,23.402657,23.255385,22.861304,22.875722,23.537318,22.383965,22.707421,...,23.597052,23.904525,23.740684,22.328488,24.180143,23.674380,23.693306,23.194393,23.418491,23.241828
2,30.00,74.75,23.054161,25.170250,24.981491,24.496934,24.615631,25.595016,24.292014,24.233603,...,25.388073,25.568388,25.159264,23.887280,25.939220,25.387846,25.692841,24.972548,25.008839,24.596393
3,30.00,75.00,22.892178,25.003719,24.834241,24.304428,24.482862,25.441842,24.080250,24.101059,...,25.194468,25.426449,25.052097,23.733018,25.804889,25.229370,25.518824,24.807497,24.838847,24.483811
4,30.00,75.25,22.449356,24.507631,24.359121,23.838091,23.968899,24.880342,23.577837,23.681398,...,24.707511,24.958772,24.650361,23.299063,25.295115,24.768298,24.992627,24.322387,24.397749,24.107134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,32.00,75.00,18.078937,20.046645,19.614374,18.795204,18.780391,19.947808,19.245566,19.935117,...,20.156663,20.426828,20.437692,19.062365,20.376230,20.193189,20.367155,19.910701,20.141071,20.386807
61,32.00,75.25,16.052225,17.943380,17.464985,16.710553,16.681903,17.826780,17.147402,17.896313,...,18.054463,18.337104,18.415430,17.080730,18.227325,18.070100,18.266324,17.830848,18.100132,18.388927
62,32.00,75.50,14.025514,15.840116,15.315596,14.625901,14.583414,15.705751,15.049238,15.857509,...,15.952264,16.247380,16.393169,15.099095,16.078421,15.947010,16.165492,15.750995,16.059192,16.391047
63,32.00,75.75,11.998802,13.736852,13.166207,12.541250,12.484926,13.584723,12.951074,13.818704,...,13.850064,14.157656,14.370908,13.117460,13.929516,13.823920,14.064661,13.671143,14.018252,14.393167


In [133]:
conv_df.to_csv('forecast_min_temp.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'forecast_min_temp.csv'

SSP126


In [141]:
#tmin
df = pd.read_csv(
    "B:\GroundwaterRech\GithubRepo\Restart\Dataset\Final forecasted data 2020-2040\Final forecasted data 2020-2040\SSP126\\tmin.csv")
df.drop(['Unnamed: 0'], inplace=True, axis=1)
df.drop(['height'], inplace=True, axis=1)
conv_df = conv_temp(df, 2020, 2035, 'tasmin')
conv_df

year,New_Latitude,New_Longitude,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
0,29.75,75.25,25.528034,24.996873,25.305749,24.689501,24.433262,24.302374,24.505798,24.655400,25.034283,25.420456,26.347762,25.603153,25.312076,25.937138,25.249160,26.014031
1,29.75,76.00,24.285877,23.958979,24.275334,23.684482,23.623477,23.434602,23.592725,23.713271,24.227366,24.465376,25.273398,24.689602,24.381296,25.010511,24.206991,25.122200
2,30.00,74.75,26.244828,25.592516,26.025530,25.394344,25.010374,24.775993,25.030566,25.355804,25.626077,26.036590,27.058081,26.198800,25.909631,26.633708,25.850567,26.609728
3,30.00,75.00,26.068380,25.450578,25.826003,25.262171,24.875940,24.639186,24.818003,25.194191,25.503760,25.901323,26.873041,26.014346,25.753860,26.467946,25.663337,26.405274
4,30.00,75.25,25.530199,24.989603,25.358204,24.807276,24.490497,24.227966,24.402224,24.764210,25.116009,25.464671,26.385320,25.586188,25.323579,26.033920,25.218588,26.000752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,32.00,75.00,20.578445,20.686816,21.226856,20.755299,20.667319,19.049630,20.076438,20.394273,21.420485,21.122374,21.888138,21.450378,20.815456,21.738937,21.097079,22.238659
61,32.00,75.25,18.462702,18.538217,19.135083,18.702367,18.665255,17.029880,18.042350,18.366535,19.339573,19.053452,19.756340,19.391660,18.735349,19.645949,19.102613,20.141775
62,32.00,75.50,16.346958,16.389618,17.043311,16.649435,16.663192,15.010129,16.008263,16.338798,17.258662,16.984531,17.624541,17.332943,16.655241,17.552961,17.108147,18.044891
63,32.00,75.75,14.231215,14.241019,14.951539,14.596503,14.661129,12.990379,13.974175,14.311060,15.177750,14.915609,15.492743,15.274225,14.575134,15.459973,15.113682,15.948007


In [135]:
conv_df.to_csv('B:\GroundwaterRech\GithubRepo\Restart\Forecast_new\SSP126\pet_SSP126.csv\\tmin_SSP126.csv', index=False)

In [139]:
#tmax
df = pd.read_csv(
    "B:\GroundwaterRech\GithubRepo\Restart\Dataset\Final forecasted data 2020-2040\Final forecasted data 2020-2040\SSP126\\tmax.csv")
df.drop(['Unnamed: 0'], inplace=True, axis=1)
df.drop(['height'], inplace=True, axis=1)
conv_df = conv_temp(df, 2020, 2035, 'tasmax')
conv_df

year,New_Latitude,New_Longitude,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
0,29.75,75.25,37.673741,35.939291,37.726193,36.281587,36.001263,37.738813,37.131654,36.222254,36.484771,37.048542,37.903782,37.583755,37.101245,37.571370,36.191628,37.300299
1,29.75,76.00,36.289148,34.866418,36.373715,35.026451,34.860860,36.855675,35.931990,34.842393,35.489566,35.909888,36.646842,36.277319,36.158082,36.077919,34.979777,35.973308
2,30.00,74.75,38.638990,36.740161,38.639398,37.174534,36.884402,38.423965,37.989454,37.205140,37.223640,37.895861,38.776979,38.484064,37.843044,38.661820,37.049413,38.211638
3,30.00,75.00,38.477703,36.595515,38.467876,37.023582,36.733447,38.342697,37.826303,37.006802,37.106136,37.701465,38.610403,38.302696,37.712204,38.435771,36.862667,38.014858
4,30.00,75.25,37.869305,36.102727,37.873676,36.465702,36.211993,37.910837,37.293717,36.404378,36.626590,37.195442,38.049728,37.731592,37.262258,37.792677,36.327856,37.432897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,32.00,75.00,34.423356,33.066563,34.664077,33.164173,32.657594,34.876116,33.963603,32.911335,33.688549,34.272753,34.621274,34.860137,34.224969,34.607381,32.588400,34.436669
61,32.00,75.25,32.111448,30.830839,32.427968,30.988723,30.474064,32.566848,31.763193,30.730248,31.482117,32.057295,32.415208,32.659233,31.985983,32.357018,30.487154,32.219248
62,32.00,75.50,29.799541,28.595116,30.191860,28.813273,28.290534,30.257580,29.562783,28.549161,29.275685,29.841836,30.209143,30.458329,29.746997,30.106655,28.385908,30.001827
63,32.00,75.75,27.487633,26.359392,27.955752,26.637824,26.107004,27.948313,27.362372,26.368073,27.069253,27.626377,28.003077,28.257425,27.508011,27.856293,26.284662,27.784406


In [140]:
conv_df.to_csv('B:\GroundwaterRech\GithubRepo\Restart\Forecast_new\SSP126\pet_SSP126.csv\\tmax_SSP126.csv', index=False)

In [127]:
obs_data = [21.64, 23.072, 22.144, 21.556, 22.416, 22.114, 21.748, 21.106, 21.21, 21.924, 20.856,
            21.824, 22.594, 22.454, 21.566, 22.114, 23.22, 22.982, 22.984, 21.926, 22.11,
            23.348, 21.966, 22.832, 21.272, 22.12, 22.52, 21.896, 23.016
            ]
modeled_data = [22.04557902, 24.06806703, 23.76300404, 22.99754573, 22.98009298, 24.21329577, 23.20718783, 23.39335325,
                23.26778293, 22.94746369, 23.25247475, 23.71161033, 22.69527904, 24.24301214, 23.83913828,
                24.20667466, 24.08398862, 23.49419176, 23.90849052, 24.05230362, 24.32223775, 24.20228897,
                22.63480233, 24.40814021, 24.21173736, 24.36980316, 23.82385299, 23.90466514, 23.82375509
                ]
bb = mean_bias_correction(observed_data=obs_data, modeled_data=modeled_data)
calculate_nse(observed=obs_data, predicted=bb)


-1.5014420341379306
NSE: -0.2075592035453906


In [128]:
qq = quantile_mapping(observed_data=obs_data, modeled_data=modeled_data)
calculate_nse(observed=obs_data, predicted=qq)

NSE: 0.9960105488170292


## Evapotranspiration

In [157]:
def conv_pet(df, start_year, end_year, var):
    df['time'] = pd.to_datetime(df['time'])
    df['year'] = df['time'].dt.year
    df = df[df['time'].dt.year.between(start_year, end_year)]
    avg_monthly_pet = df.groupby(['lat', 'lon', 'year'])[var].mean().reset_index()
    merged_df = ldf.merge(avg_monthly_pet, on=['lon', 'lat'], how='left')
    pivot_df = merged_df.pivot(index=['New_Latitude', 'New_Longitude'], columns='year', values=var).reset_index()
    return pivot_df

In [156]:
df

,Unnamed: 0,time,lat,lon,tasmin,tasmax,tas,month,PET
0,0,2020-06-01 12:00:00,29.3,73.55,21.658688,36.707586,30.864836,6,7.381496
1,1,2020-06-01 12:00:00,29.3,73.80,21.576681,36.665179,30.706705,6,7.367184
2,2,2020-06-01 12:00:00,29.3,74.05,21.494674,36.622771,30.548573,6,7.352797
3,3,2020-06-01 12:00:00,29.3,74.30,21.412667,36.580364,30.390442,6,7.338334
4,4,2020-06-01 12:00:00,29.3,74.55,21.330660,36.537957,30.232310,6,7.323796
...,...,...,...,...,...,...,...,...,...
381883,381883,2035-10-31 12:00:00,32.3,75.30,2.782446,20.150101,11.248109,10,3.257633
381884,381884,2035-10-31 12:00:00,32.3,75.55,0.398122,17.820745,9.007311,10,3.011090
381885,381885,2035-10-31 12:00:00,32.3,75.80,-1.986202,15.491389,6.766512,10,2.763745
381886,381886,2035-10-31 12:00:00,32.3,76.05,-4.370527,13.162033,4.525714,10,2.515601


In [158]:
df = pd.read_csv(
    "B:\GroundwaterRech\GithubRepo\Restart\Dataset\Final forecasted data 2020-2040\Final forecasted data 2020-2040\SSP126\\pet.csv")
conv_df = conv_pet(df, 2020, 2035, 'PET')
conv_df

year,New_Latitude,New_Longitude,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
0,29.75,75.25,5.946657,5.449734,5.959938,5.530943,5.541001,6.108348,5.925880,5.573211,5.642155,5.759931,5.869256,5.824260,5.789270,5.770164,5.422567,5.647477
1,29.75,76.00,5.755143,5.313389,5.733773,5.326935,5.345827,6.000980,5.738531,5.328153,5.490194,5.592815,5.689679,5.599135,5.669523,5.483620,5.242404,5.406439
2,30.00,74.75,6.115045,5.594527,6.117545,5.688829,5.715683,6.237000,6.093099,5.747861,5.766640,5.912760,6.011574,5.997057,5.916430,5.987304,5.583761,5.825503
3,30.00,75.00,6.097044,5.574817,6.099409,5.665696,5.691001,6.237206,6.082574,5.715203,5.751844,5.877167,5.992570,5.972498,5.902739,5.946573,5.557522,5.801428
4,30.00,75.25,6.011416,5.510383,6.002557,5.577051,5.604459,6.180616,5.998146,5.611646,5.678306,5.805357,5.913364,5.878419,5.843176,5.829674,5.476745,5.698113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,32.00,75.00,5.867669,5.440914,5.807581,5.322401,5.257420,6.156223,5.761144,5.353928,5.514215,5.729288,5.735169,5.775245,5.746664,5.678841,5.122755,5.526254
61,32.00,75.25,5.547072,5.163456,5.509690,5.057496,4.978762,5.809411,5.461584,5.081174,5.234768,5.435463,5.453731,5.484327,5.446640,5.383397,4.873210,5.250053
62,32.00,75.50,5.229610,4.886964,5.213736,4.793267,4.702584,5.467160,5.163893,4.809361,4.956788,5.143729,5.173187,5.195108,5.148783,5.090359,4.624228,4.975254
63,32.00,75.75,4.915362,4.611551,4.919813,4.529897,4.429011,5.129561,4.868248,4.538829,4.680390,4.854168,4.893634,4.907734,4.853248,4.799823,4.376017,4.701988


In [159]:
conv_df.to_csv("B:\GroundwaterRech\GithubRepo\Restart\Forecast_new\SSP126\pet_SSP126.csv")

### TEST

In [13]:
l = result[result['lon'] == 74.80]

In [14]:
l1 = l[l['lat'] == 31.80]

In [15]:
l1

,lat,lon,year,pr
3625,31.8,74.8,1986,54.227091
3626,31.8,74.8,1987,43.378693
3627,31.8,74.8,1988,35.255272
3628,31.8,74.8,1989,34.383029
3629,31.8,74.8,1990,40.643300
3630,31.8,74.8,1991,37.851230
3631,31.8,74.8,1992,38.467825
3632,31.8,74.8,1993,81.933598
3633,31.8,74.8,1994,58.695271
3634,31.8,74.8,1995,36.223168


In [23]:
modeled_data = np.array(l1['pr'])
obs_data = np.array([110.96,
                     62.08,
                     173.2,
                     119.02,
                     152.28,
                     88.04,
                     124.04,
                     97.26,
                     100.66,
                     163.44,
                     149.88,
                     146.08,
                     98.06,
                     86.42,
                     109,
                     122.32,
                     91.02,
                     119.3,
                     96.38,
                     106.68,
                     188.04,
                     96.76,
                     144.64,
                     64.86,
                     137.9,
                     148.6,
                     115.18,
                     156.74,
                     130.16,
                     ])
